# Dynamic Programming with Markov Matrices
## Andres Perez (Worked with group)
(10/5) 5 extra points

A company is considering whether to make an ad buy.  The buy costs \\$1 million.  

In a good year, the company has sales of \\$4 million; in a normal year, its sales are \\$2 million; and in a bad year, its sales are only \\$1 million.  The company thinks (rightly) that an ad buy is akin to investing in its future, and it has an internal rate of return on other projects at gross rate $1+r$.

If there is ***no ad buy***, the Markov matrix describing transitions between states is:

$$M(0) = 
\begin{bmatrix}
0.8 & 0.2 & 0\\
0.2 & 0.8 & 0 \\
0.1 & 0.4 & 0.5
\end{bmatrix}$$

If there is ***an ad buy***, the Markov matrix describing transitions between states is:

$$M(1) = 
\begin{bmatrix}
0.5 & 0.5 & 0\\
0.1 & 0.6 & 0.3 \\
0.1 & 0.3 & 0.6
\end{bmatrix}$$

In these matrices, the first row represents a bad year, the second a normal year, and the third a good year.  The columns are likewise.

#### Question 1
#### What are the five elements of a dynamic program, as they pertain to this problem?

1. The State Space is 

$$
X = \{\textrm{Bad Year, Normal Year, Good Year} \}
$$

2. Control Set 

$$
U = \{\textrm{No Ad Buy, Ad Buy} \}
$$

3. Reward Function 

$$r: X \times U \rightarrow \mathbb{R}$$

|  | No Ad Buy  |  Ad Buy   |
|--:-:----|------|------|
|Bad Year     |1   |0     |
|Normal Year  |2     |1     |
|Good Year |4 |3 |

4. Transition Rule

$$M(\textrm{No Ad Buy}) = 
\begin{bmatrix}
0.8 & 0.2 & 0\\
0.2 & 0.8 & 0 \\
0.1 & 0.4 & 0.5
\end{bmatrix}$$

and

$$M(\textrm{Ad Buy}) = 
\begin{bmatrix}
0.5 & 0.5 & 0\\
0.1 & 0.6 & 0.3 \\
0.1 & 0.3 & 0.6
\end{bmatrix}$$


5. Discount Factor 

$$ \beta = \frac{1}{1+r}$$

#### Question 2
#### Write the Bellman equation, when the company is having a normal year.

V = Value

$$V(\mathrm{Normal}) = \mathrm{max}\{ 2 + \beta[0.2\mathrm{V(Bad)} + 0.8\mathrm{V(Normal)}], 1 + \beta[0.1\mathrm{V(Bad)} + 0.6\mathrm{V(Normal)} + 0.3\mathrm{V(Good)}]\}$$

#### Question 3
#### Import numpy, pandas, and quantecon. Define the States and Controls as lists. Define the reward as a numpy array. Set 𝛽=0.9. Define the value function as an array of zeros, one for each state.

In [1]:
import numpy as np
import pandas as pd
import quantecon as qe
States = ['Bad', 'Normal', 'Good']
Controls = ['No Buy', 'Buy']
reward= np.array([[1,0],[2,1],[4,3]])
beta = 0.9
value = np.zeros(len(States))

#### Question 4
#### Make a dictionary called transition that has two entries, indexed by 'No Buy' and 'Buy'. The entries are the two Markov matrices.

In [2]:
M0 = np.array([[0.8,0.2,0],[0.2,0.8,0],[0.1,0.4,0.5]])
M1 = np.array([[0.5,0.5,0],[0.1,0.6,0.3],[0.1,0.3,0.6]])
transition = {'No Buy': M0, 'Buy': M1}

#### Question 5
#### Write a function that checks that the value function, the reward function, the transition rule, and the discount factor all make sense.

In [3]:
def check_parameters(States,Controls,reward,transition,value,beta):
    n = len(States)
    m = len(Controls)
    if len(value) != n: print("The value function and the state space are not comformable.")
    if reward.shape != (n,m): print("The reward function is not well defined.")
    if len(transition) != m:
        print("There is not the requisite number of transition matrices.")
    if not all(transition[control].shape == (n,n) for control in Controls):
        print("At least one of the Markov transitions is one of the wrong dimension.")
    if not all(np.ones(n)@transition[control]@np.ones(n) == n for control in Controls):
        print("Give me a break! One of the transitions matrices is not Markov")
    if not all((transition[control] >= 0).all() and (transition[control] <= 1).all() for control in Controls):
        print("Give me a break! The elements of the transition matrices are not probabilities.")
    if beta < 0 or beta >= 1:
        print("Something is wrong with the discount factor.")
    return (print("T have done the requisite checks."))

#### Question 6
#### Create a function that takes as inputs the state, the control, and the transition. It gives as an output the relevant transition probabilities.

In [4]:
def next_state_probs(state,control, transition): 
    return(transition[control][States.index(state)])

#### Question 7
#### Write a function that updates value function, using the Bellman relation

In [5]:
def next_value(States,Controls,reward, transition, value, beta): 
    v_next =[]
    for state in States: 
        bellman = []
        for control in Controls: 
            bellman.append(reward[States.index(state),Controls.index(control)] + beta*(value@next_state_probs(state,control,transition)))
        v_next.append(max(bellman))
    return (v_next)

#### Question 8
#### Write a function that iterates on the value function to solve the dynamic program.

In [6]:
def solve(States,Controls,reward,transition,value,beta): 
    check = 1.0
    epsilon = 10e-5
    max_iter = 100
    iter = 0 
    while check > epsilon: 
        v1 = np.array(next_value(States,Controls,reward,transition,value,beta))
        check = max(abs(value-v1))
        value = v1
        iter += 1 
        if(iter > max_iter): 
            print('Algorithm did not converge')
            break 
    return(value)
    
value = solve(States,Controls,reward,transition,value,beta)

#### Question 9
#### Write a function that records the optimal policy

In [7]:
def optimal_policy(States,Controls,reward,transition,value,beta): 
    policy =[]
    for state in States: 
        z=[]
        for control in Controls:
            z.append(reward[States.index(state),Controls.index(control)] + beta*(value@next_state_probs(state,control,transition)))
        policy.append(Controls[z.index(max(z))])
    return(policy)

policy = optimal_policy(States,Controls,reward,transition,value,beta)

#### Question 10
#### Write a function that records the transition matrix, when the agent is following the optimal policy. Then compute its ergodic distribution.

In [8]:
def M_Optimal(States,transition,policy):
    M = np.empty_like(list(transition.items())[0][1])
    for state in States:
        M[States.index(state),:] = transition[policy[States.index(state)]][States.index(state),:]
    return(M)

P = M_Optimal(States,transition,policy)
from quantecon import MarkovChain 
mc = qe.MarkovChain(P)
p_stationary = mc.stationary_distributions 

#### Question 11
#### Display the output as a Panda DataFrame, whose rows are indexed by the state.  The five columns should give: (1) reward if 'No buy'; (2) reward if 'Buy'; (3) the solved value function; (4) the optimal policy; and (5) the probabilities from the ergodic distribution of the states.

In [9]:
PS5 = pd.DataFrame(reward,States,Controls)
PS5['Value Function'] = value
PS5['optimal Policy'] = policy
PS5['Probabilities'] = p_stationary[0]
PS5

,No Buy,Buy,Value Function,optimal Policy,Probabilities
Bad,1,0,14.804697,No Buy,0.333333
Normal,2,1,17.474440,Buy,0.416667
Good,4,3,21.132976,No Buy,0.250000


#### Question 12
#### Write a class called DP_Markov() that incorporates everything you have done here.  If you succeed at this task, I will give five points extra credit to everyone in your group. 

In [10]:
value = np.zeros(len(States))

class DP_Markov:
    
    def __init__(self, states, controls, reward, transition, value, beta=0.9):
        n = len(States)
        m = len(Controls)
        if len(value) != n:
            raise ValueError("The value function and the state space are not comformable.")
        if reward.shape != (n,m): 
            raise ValueError("The reward function is not well defined.")
        if len(transition) != m:
            raise ValueError("There is not the requisite number of transition matrices.")
        if not all(transition[control].shape == (n,n) for control in Controls):
            raise ValueError("At least one of the Markov transitions is one of the wrong dimension.")
        if not all(np.ones(n)@transition[control]@np.ones(n) == n for control in Controls):
            raise ValueError("Give me a break! One of the transitions matrices is not Markov")
        if not all((transition[control] >= 0).all() and (transition[control] <= 1).all() for control in Controls):
            raise ValueError("Give me a break! The elements of the transition matrices are not probabilities.")
        if beta < 0 or beta >= 1:
            raise ValueError("Something is wrong with the discount factor.")
        self.states = states
        self.controls = controls
        self.reward = reward
        self.transition = transition
        self.value = value
        self.beta = beta
    
    def next_state_probs(self, state, control):    
        return(self.transition[control][self.states.index(state)])
    
    def next_value(self): 
        v_next =[]
        for state in self.states: 
            bellman = []
            for control in self.controls: 
                bellman.append(self.reward[self.states.index(state),self.controls.index(control)] + self.beta*(self.value@self.next_state_probs(state, control)))
            v_next.append(max(bellman))
        return (v_next)
    
    def solve(self): 
        check = 1.0
        epsilon = 10e-5
        max_iter = 100
        iter = 0 
        while check > epsilon: 
            v1 = np.array(self.next_value())
            check = max(abs(self.value-v1))
            self.value = v1
            iter += 1 
            if(iter > max_iter): 
                print('Algorithm did not converge')
                break 
    
    def optimal_policy(self): 
        policy =[]
        for state in self.states: 
            z=[]
            for control in self.controls:
                z.append(self.reward[self.states.index(state),self.controls.index(control)] + self.beta*(self.value@self.next_state_probs(state, control)))
            policy.append(Controls[z.index(max(z))])
        return(policy)
    
    def M_Optimal(self):
        M = np.empty_like(list(self.transition.items())[0][1])
        for state in self.states:
            M[self.states.index(state),:] = self.transition[self.optimal_policy()[self.states.index(state)]][self.states.index(state),:]
        return(M)
    
    def display_optimal(self):
        self.solve()
        df = pd.DataFrame(self.reward,self.states,self.controls)
        P = self.M_Optimal()
        mc = qe.MarkovChain(P)
        p_stationary = mc.stationary_distributions 
        df['Value Function'] = self.value
        df['optimal Policy'] = self.optimal_policy()
        df['Probabilities'] = p_stationary[0]
        return(df)

DP_Markov(States,Controls,reward, transition, value, beta).display_optimal()

,No Buy,Buy,Value Function,optimal Policy,Probabilities
Bad,1,0,14.804697,No Buy,0.333333
Normal,2,1,17.474440,Buy,0.416667
Good,4,3,21.132976,No Buy,0.250000
